# Donkey Car FPS Logger

## Frame per second logger

What does a frame per second do? It simply means that how many times we have run within one second.

Consider the Hello World program and the `rate_hz` parameter, is it possible to output 10,000 lines of `Hello World` if you have set `rate_hz` to 10,000?

Let's try

In [ ]:
import donkeycar as dk
import time

class HelloWorld:
    def __init__(self):
        self.current_time = None
        
        pass
        
    def run(self):
        previous_timestamp = self.current_time
        self.current_time = time.time()
        # print(f"last_time = \t {previous_timestamp}")
        # print(f"current_time = \t {self.current_time}" )

        if previous_timestamp is not None:
            difference = self.current_time - previous_timestamp
            # print(f"Difference = {difference}")
            loop_hz = 1 / difference
            print(f"Running {loop_hz} per second\n")

class SlowPart():
    def __init__(self):
        pass

    def run(self):
        time.sleep(2)
        
V = dk.vehicle.Vehicle()

V.add(camera())
V.add(FinishLineDetector())

V.add(HelloWorld())
# V.add(SlowPart())

V.start(rate_hz=50000)

Do you think it is really running at 10,000 times per second?

Obviously not. How do we know exactly how much time it has run? Let's write a program

In [39]:

def add_camera(V, cfg, camera_type):
    """
    Add the configured camera to the vehicle pipeline.

    :param V: the vehicle pipeline.
              On output this will be modified.
    :param cfg: the configuration (from myconfig.py)
    """
    logger.info("cfg.CAMERA_TYPE %s"%cfg.CAMERA_TYPE)
    if camera_type == "stereo":
        if cfg.CAMERA_TYPE == "WEBCAM":
            from donkeycar.parts.camera import Webcam

            camA = Webcam(image_w=cfg.IMAGE_W, image_h=cfg.IMAGE_H, image_d=cfg.IMAGE_DEPTH, iCam = 0)
            camB = Webcam(image_w=cfg.IMAGE_W, image_h=cfg.IMAGE_H, image_d=cfg.IMAGE_DEPTH, iCam = 1)

        elif cfg.CAMERA_TYPE == "CVCAM":
            from donkeycar.parts.cv import CvCam

            camA = CvCam(image_w=cfg.IMAGE_W, image_h=cfg.IMAGE_H, image_d=cfg.IMAGE_DEPTH, iCam = 0)
            camB = CvCam(image_w=cfg.IMAGE_W, image_h=cfg.IMAGE_H, image_d=cfg.IMAGE_DEPTH, iCam = 1)
        else:
            raise(Exception("Unsupported camera type: %s" % cfg.CAMERA_TYPE))

        V.add(camA, outputs=['cam/image_array_a'], threaded=True)
        V.add(camB, outputs=['cam/image_array_b'], threaded=True)

        from donkeycar.parts.image import StereoPair

        V.add(StereoPair(), inputs=['cam/image_array_a', 'cam/image_array_b'],
            outputs=['cam/image_array'])
        if cfg.BGR2RGB:
            from donkeycar.parts.cv import ImgBGR2RGB
            V.add(ImgBGR2RGB(), inputs=["cam/image_array_a"], outputs=["cam/image_array_a"])
            V.add(ImgBGR2RGB(), inputs=["cam/image_array_b"], outputs=["cam/image_array_b"])

    elif cfg.CAMERA_TYPE == "D435":
        from donkeycar.parts.realsense435i import RealSense435i
        cam = RealSense435i(
            enable_rgb=cfg.REALSENSE_D435_RGB,
            enable_depth=cfg.REALSENSE_D435_DEPTH,
            enable_imu=cfg.REALSENSE_D435_IMU,
            device_id=cfg.REALSENSE_D435_ID)
        V.add(cam, inputs=[],
              outputs=['cam/image_array', 'cam/depth_array',
                       'imu/acl_x', 'imu/acl_y', 'imu/acl_z',
                       'imu/gyr_x', 'imu/gyr_y', 'imu/gyr_z'],
              threaded=True)
    else:
        inputs = []
        outputs = ['cam/image_array']
        threaded = True
        cam = get_camera(cfg)
        if cam:
            V.add(cam, inputs=inputs, outputs=outputs, threaded=threaded)
        if cfg.BGR2RGB:
            from donkeycar.parts.cv import ImgBGR2RGB
            V.add(ImgBGR2RGB(), inputs=["cam/image_array"], outputs=["cam/image_array"])

Let's introduce an important library first. It is called `time`

In [ ]:
import time

It has a function called `time()` which return a timestamp. Google what a timestamp is.

In [ ]:
time.time()

## Challenge

Now, use the following scaffold and write a program to calculate the fps

In [ ]:
class FrequencyLogger(object):
    """
    Log current, min and max of frequency value
    """

    def __init__(self):
        pass

    def run(self):
        pass



## Shutdown

In [ ]:

    def shutdown(self):
        if self.fps_list:
            logger.info(f"fps (min/max) = {min(self.fps_list):2d} / {max(self.fps_list):2d}")
            logger.info(f"fps list = {self.fps_list}".format())